In [1]:
import pandas as pd
import numpy as np
import nanoid
import pymongo
from openai import OpenAI
import requests
from datetime import datetime



In [15]:
quantites = np.random.randint(1,100,size=48)
prices_buy = np.random.randint(1,2,size=48)*10 + np.random.randint(0,10,size=48) + np.random.randint(0,10,size=48)*0.1
prices_sale = prices_buy - np.random.randint(0,10,size=48) + np.random.randint(0,10,size=48)*0.1


In [18]:
data = pd.read_csv('data/flowershopdata.csv')
data['Description'] = data[['Description', 'Needs','Season']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
data['Buy_price'] = prices_buy
data['Sale_price'] = np.round(prices_sale,2)
data['Quantities'] = quantites
jpg_paths = [f"init_data/products/img_flowers/{flower.lower()}.jpg" for flower in data['Name']]
data['pic_path'] = jpg_paths
data['id'] = [nanoid.generate(size=10) for x in range(48)]
data['date'] =[datetime.now().strftime("%d-%m-%Y %H:%M:%S") for x in range(48)]
data['tags'] = ['flower' for x in range(48)]
data.drop(columns=['Needs','Season'],inplace=True)

In [19]:
data.to_csv('data/flowershopdata_clean.csv')

In [46]:
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def fetch_first_image_url(query):
    options = webdriver.SafariOptions()
    driver = webdriver.Safari(options=options)

    search_url = f"https://pl.pinterest.com/search/pins/?q={query}%20flower&rs=typed"
    driver.get(search_url)

    time.sleep(2)
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'img')

    if len(image_elements) >= 10:
        # Get the URL of the 10th image
        image_element = image_elements[9]  # Index 9 corresponds to the 10th image (0-based index)
        image_url = image_element.get_attribute('src')
    else:
        print("Not enough images found on the page")
        image_url = None

    driver.quit()
    return image_url

def save_image_from_url(url, directory, filename):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image_path = os.path.join(directory, filename)
    image.save(image_path)
    print(f"Image saved to {image_path}")



In [6]:
import nanoid
def create_product(product_data):
    base_url = "http://127.0.0.1:8000"
    response = requests.post(f"{base_url}/products/", json=product_data)
    print(response.json())

In [7]:
data = pd.read_csv('data/flowershopdata_clean.csv').drop(columns=['Unnamed: 0'])



In [40]:
client = QdrantClient(url="http://127.0.0.1:6333")
names = [item.name for item in client.get_collections().collections]
print(names)

[]


In [32]:
# from sentence_transformers import SentenceTransformer, util
from qdrant_client import QdrantClient, AsyncQdrantClient
from qdrant_client.models import Distance, VectorParams
from qdrant_client.models import PointStruct

# model = SentenceTransformer('intfloat/e5-small-v2')
vec_len = 384
client = QdrantClient(url="http://127.0.0.1:6333")
names = [item.name for item in client.get_collections().collections]
if 'products_description' not in names:
    client.create_collection(
        collection_name="products_description",
        vectors_config=VectorParams(size=vec_len, distance=Distance.COSINE),)
else:
    print('Collection exits')


In [39]:
client.delete_collection("products_description")

False

In [8]:
points = [PointStruct(id=index, 
                      vector=model.encode(item['Description'], normalize_embeddings=True), 
                      payload={"Id": item['id'],
                               "Name": item['Name'],
                               "Description":item['Description']}) for index,item in data.iterrows()]


In [215]:
operation_info = client.upsert(
    collection_name="products_description",
    wait=True,
    points=points,
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [35]:
from pymongo import MongoClient
from openai import OpenAI
from typing import List
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient


In [88]:
question = "I would like white flower that needs acidic soil"
model = SentenceTransformer("intfloat/e5-small-v2")
bot = {
    "mongo_client": MongoClient("mongodb://localhost:27017/"),
    "qdrant_client": QdrantClient("http://localhost:6333"),
    "openai_client":  OpenAI(api_key="sk-proj-8j1NyU5gvA4njucbGG3uT3BlbkFJxBLHcRn4z7futfYb4sgb")
},

In [5]:
client = AsyncQdrantClient("http://localhost:6333")

NameError: name 'AsyncQdrantClient' is not defined

In [119]:
requests.post("http://127.0.0.1:8000/products/vec_search/", json={'question': 'Red flower'}).json()


[{'id': 6,
  'version': 6,
  'score': 0.81549656,
  'payload': {'Description': 'Named for its red colored sap that flows from cut stems, bloodroot flowers are white with a bright yellow center. Needs well-drained soil. Blooms in early spring.',
   'Id': 'AtfxNCsTNK',
   'Name': 'Bloodroot'},
  'vector': None,
  'shard_key': None},
 {'id': 21,
  'version': 21,
  'score': 0.8153006,
  'payload': {'Description': 'Small, bell-shaped flowers, often used as ground cover and in rock gardens, known for their resilience and beauty. Needs acidic soil. Blooms in late summer to fall.',
   'Id': 'CS9fWn1prj',
   'Name': 'Heather'},
  'vector': None,
  'shard_key': None},
 {'id': 27,
  'version': 27,
  'score': 0.8136447,
  'payload': {'Description': 'Tall spikes of flowers that resemble a dragon’s mouth when squeezed, available in a wide range of colors. Needs well-drained soil. Blooms in spring to fall.',
   'Id': 'A7y3--teGu',
   'Name': 'Snapdragon'},
  'vector': None,
  'shard_key': None}]

In [38]:
count = client.count(collection_name="products_description")
count.count

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `products_description` doesn\'t exist!"},"time":0.000152104}'

In [22]:
# Connect to the MongoDB server
client_mongo = pymongo.MongoClient("mongodb://localhost:27017/")

# Select your database
db = client_mongo["ecommerce-app"]

# List all collection names in the database
collection_names = db.list_collection_names()


collection = db['products']
# Find all documents in the collection
documents = collection.find()
for x in documents:
    print(x)


# print(len(documents))




In [41]:
db.drop_collection('products')
for document in documents:
    print(document)

In [135]:
qdrant_client = QdrantClient('localhost:6333')
results = qdrant_client.search(
    collection_name="products_description",
    query_vector=list(model.encode('dwa', normalize_embeddings=True)),
    limit=3,
)
type(results[0])

qdrant_client.http.models.models.ScoredPoint

In [52]:
model[1].word_embedding_dimension

384

In [20]:
count = client.count(collection_name='products_description')

print(count)

count=0


In [23]:
'/dwa/trzy.jpg'.split('.')

['/dwa/trzy', 'jpg']